In [1]:
import numpy as np

In [2]:
def forward_predictor(data,window_width):
    """
    Calculate the forward prediction based on data. https://doi.org/10.1016/0165-0270(91)90118-J
    """
    output = [data[0]] # Leere Liste wird erstellt, da wir diese nach und nach befüllen.
    for i in range(1,len(data)): # range gibt uns nach und nach die Indices [0, N-1], wo N die Länge der Daten ist. 
        if i-window_width>=0: 
            prediction = 1/window_width*np.sum(data[i-window_width:i])
        else:
            prediction = 1/i*np.sum(data[:i])
        output.append(prediction)
    return output

In [3]:
def backward_predictor(data,window_width):
    """
    Calculate the backward prediction based on data. https://doi.org/10.1016/0165-0270(91)90118-J
    """
    output = list()
    len_data = len(data)
    for i in range(len_data-1):
        if i+window_width<len_data: 
            prediction = 1/window_width*np.sum(data[i+1:i+1+window_width])
        else:
            prediction = 1/(len_data-i-1)*np.sum(data[i+1:])
        output.append(prediction)
    output.append(data[-1])
    return output

In [4]:
def calculate_forward_weights(data, M, forward_prediction, p):
    """
    Calculate the weight of the forward predictor.
    """
    #predictors should be given as lists, if the first element of forward_prediction is not a list
    #it is assumed that only a single forward prediction was given
    if type(forward_prediction[0]) is not list:
        forward_prediction=[forward_prediction]
    
    len_data = len(data)
    n_predictors = len(forward_prediction)
    #allocate weight array
    f = np.zeros((n_predictors,len_data))

    for i, prediction in enumerate(forward_prediction):
        for k in range(len_data):
            for j in range(M):
                if not k-j<0:
                    f[i,k] += (data[k-j]-prediction[k-j])**2
            #zero should only occur at the beginning, in order to avoid infinities it is replaced by one
            if f[i,k]==0: f[i,k]=1
            f[i,k] = f[i,k]**-p
    return f

In [5]:
def calculate_backward_weights(data, M, backward_prediction, p):
    """
    Calculate the weight of the forward predictor.
    """
    #predictors should be given as lists, if the first element of backward_prediction is not a list
    #it is assumed that only a single backward prediction was given
    if type(backward_prediction[0]) is not list:
        backward_prediction=[backward_prediction]
    
    len_data = len(data)
    n_predictors = len(backward_prediction)
    #allocate weight array
    b = np.zeros((n_predictors,len_data))

    for i, prediction in enumerate(backward_prediction):
        for k in range(len_data):
            for j in range(M):
                if not k+j>=len_data:
                    b[i,k] += (data[k+j]-prediction[k+j])**2
            #zero should only occur at the beginning, in order to avoid infinities it is replaced by one
            if b[i,k]==0: b[i,k]=1
            b[i,k] = b[i,k]**-p
    return b

In [14]:
x = np.array([1,2,3,5,6,8,10,12,14,20,10,15,2,3,5,18,14,12,14])

tp2 = 1/2*np.array([2,2,3,5,8,11,14,18,22,26,34,30,25,17,5,8,23,32,26])

tp3 = 1/3*np.array([3,3,4,6,10,14,19,24,30,36,46,44,45,27,20,10,26,37,44])

In [20]:
def test_forward_pred():
    #input
    x = np.array([1,2,3,5,6,8,10,12,14,20,10,15,2,3,5,18,14,12,14])
    #targets
    tp2 = 1/2*np.array([2,2,3,5,8,11,14,18,22,26,34,30,25,17,5,8,23,32,26])
    tp3 = 1/3*np.array([3,3,4,6,10,14,19,24,30,36,46,44,45,27,20,10,26,37,44])
    p2 = forward_predictor(x,2)
    p3 = forward_predictor(x,3)
    assert np.all(tp2==p2)
    assert np.all(tp3==p3)

In [21]:
test_forward_pred()

AssertionError: 

In [7]:
pred_2 = forward_predictor(x,2)
print(pred_2,x)

[1, 1.0, 1.5, 2.5, 4.0] [ 1  2  3  5 10]


In [ ]:
t_out = [
        1,
        1,
        
        ]